<a href="https://colab.research.google.com/github/BimalGhimire38/Extra_Python_Stuffs/blob/main/Extracting_English_lessons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing for IELTS
2082/08/27  07:38 AM

In [1]:
!pip install requests beautifulsoup4 fpdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 19.1 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time

def get_topics(main_url):
    """
    Scrape the main page to get list of topics and their URLs.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(main_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    ol = soup.find('ol')
    if not ol:
        raise ValueError("Could not find the ordered list of topics.")

    topics = []
    for li in ol.find_all('li'):
        a = li.find('a')
        if a:
            title = a.get_text().strip()
            url = a['href']
            if not url.startswith('http'):
                url = 'https://basicenglishspeaking.com' + url
            topics.append((title, url))

    print(f"Found {len(topics)} topics.")
    return topics

def get_lesson_content(lesson_url):
    """
    Scrape a single lesson page to extract question-answer pairs.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(lesson_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    content_div = soup.find('div', class_='entry-content')
    if not content_div:
        raise ValueError(f"Could not find content div for {lesson_url}")

    paragraphs = content_div.find_all('p')
    questions_answers = []

    # Assume pairs: odd index question, even answer (0-based)
    for i in range(0, len(paragraphs), 2):
        if i + 1 < len(paragraphs):
            question = paragraphs[i].get_text().strip()
            # Filter out empty or non-question paragraphs
            if question.endswith('?') and len(question) > 10:
                answer = paragraphs[i + 1].get_text().strip()
                if answer and not answer.endswith('?'):
                    questions_answers.append((question, answer))

    return questions_answers

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    """
    Generate a single PDF with all lessons.
    """
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)

    cover_title = "75 Daily English Conversation Lessons"
    pdf.cell(0, 10, txt=cover_title, ln=True, align='C')
    pdf.ln(10)

    pdf.set_font('Arial', '', 12)

    lesson_count = 0
    for title, url in topics:
        lesson_count += 1
        print(f"Processing lesson {lesson_count}: {title}")

        # Add title with bold
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 10, txt=title, ln=True, align='C')
        pdf.ln(5)

        # Reset to normal font
        pdf.set_font('Arial', '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            for q, a in qa_pairs:
                # Question bold
                pdf.set_font('Arial', 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")

                # Answer normal
                pdf.set_font('Arial', '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)

            # Page break after every 5 lessons or if page is full
            if lesson_count % 5 == 0:
                pdf.add_page()

        except Exception as e:
            print(f"Error processing {title}: {e}")
            pdf.set_font('Arial', '')
            pdf.cell(0, 10, txt=f"Error loading content for {title}", ln=True)

        time.sleep(1)  # Be polite to the server

    pdf.output(output_file)
    print(f"PDF generated: {output_file}")

if __name__ == "__main__":
    main_url = "https://basicenglishspeaking.com/daily-english-conversation-topics/"
    topics = get_topics(main_url)
    create_pdf(topics)

ValueError: Could not find the ordered list of topics.

In [3]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import time
import re

def get_topics(main_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(main_url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <a> tags that look like lesson links (href ends with /word-or-phrase/)
    lesson_links = soup.find_all('a', href=re.compile(r'/[a-zA-Z-]+$'))

    topics = []
    seen = set()  # Avoid duplicates
    for a in lesson_links:
        url = a['href']
        if not url.startswith('http'):
            url = 'https://basicenglishspeaking.com' + url if url.startswith('/') else 'https://basicenglishspeaking.com/' + url
        title = a.get_text().strip()
        if title and url not in seen and len(title) > 1:
            topics.append((title, url))
            seen.add(url)

    # Sort by approximate number if possible (fallback to as-is)
    def sort_key(item):
        text = item[0] + ' ' + item[1]
        numbers = re.search(r'(\d+)', text)
        return int(numbers.group(1)) if numbers else 999

    topics.sort(key=sort_key)

    print(f"Found {len(topics)} topics.")
    if len(topics) < 70:
        print("Warning: Fewer topics found than expected. Site may have changed.")
    return topics

def get_lesson_content(lesson_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(lesson_url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        raise ValueError(f"Failed to load {lesson_url}: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')
    content_div = soup.find('div', class_='entry-content')
    if not content_div:
        raise ValueError(f"Could not find content div for {lesson_url}")

    paragraphs = content_div.find_all('p')
    questions_answers = []

    i = 0
    while i < len(paragraphs) - 1:
        q_para = paragraphs[i].get_text(separator=' ').strip()
        a_para = paragraphs[i+1].get_text(separator=' ').strip()

        if q_para.endswith('?') and len(q_para) > 10 and a_para and not a_para.endswith('?'):
            questions_answers.append((q_para, a_para))
            i += 2
        else:
            i += 1  # Skip if not a clear Q-A pair

    return questions_answers

def create_pdf(topics, output_file='all_english_lessons.pdf'):
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font('Arial', '', 12)

    lesson_count = 0
    for title, url in topics:
        lesson_count += 1
        print(f"Processing lesson {lesson_count}: {title} ({url})")

        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 10, txt=f"{lesson_count}. {title}", ln=True, align='C')
        pdf.ln(5)

        pdf.set_font('Arial', '', 11)

        try:
            qa_pairs = get_lesson_content(url)
            if not qa_pairs:
                pdf.multi_cell(0, 6, "No Q&A content found.")
            for q, a in qa_pairs:
                pdf.set_font('Arial', 'B')
                pdf.multi_cell(0, 6, f"Q: {q}")

                pdf.set_font('Arial', '')
                pdf.multi_cell(0, 6, f"A: {a}")
                pdf.ln(3)

            if lesson_count % 4 == 0:  # Add page break every few lessons to avoid overflow
                pdf.add_page()

        except Exception as e:
            print(f"Error processing {title}: {e}")
            pdf.set_font('Arial', 'I')
            pdf.multi_cell(0, 6, f"Error loading content: {str(e)}")

        time.sleep(1.5)  # Increased delay to be extra polite to the server

    pdf.output(output_file)
    print(f"PDF generated successfully: {output_file}")

if __name__ == "__main__":
    main_url = "https://basicenglishspeaking.com/daily-english-conversation-topics/"
    topics = get_topics(main_url)
    create_pdf(topics)

Found 0 topics.
PDF generated successfully: all_english_lessons.pdf


/tmp/ipython-input-3847116682.py:77: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/tmp/ipython-input-3847116682.py:78: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
/tmp/ipython-input-3847116682.py:78: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt="75 Daily English Conversation Lessons", ln=True, align='C')
/tmp/ipython-input-3847116682.py:80: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
